# Stenoprotokoly - příprava dat
Web poslanecké sněmovny má velký archív stenoprotokolů, můžete si třeba pročíst, jak se probíraly záležitosti [v roce 1920](http://www.psp.cz/eknih/1920ns/index.htm). Nás bude zajímat současná sněmovna, k níž stena [najdeme tadyk](http://www.psp.cz/eknih/2013ps/stenprot/index.htm). Naštěstí nemusíme data stahovat přímo v tomto formátu, PSP nabízí i zazipovaný archív, [konkrétně tady](http://www.psp.cz/eknih/2013ps/stenprot/zip/index.htm).

Probíhat to bude asi takto: stáhneme zipy, rozbalíme, rozparsujeme do jsonů, nasypeme do skladiště dle libosti. Bude třeba Python 3 a pár balíků. Většina ve standardní knihovně, mimo ní používám akorat PyQuery.

Je třeba dodělat:
- datum projevu
- přidat více Sněmoven
- rozparsovat jméno řečníka

In [122]:
import urllib.request as ur
import urllib.parse as up
# from pyquery import PyQuery as pq
import lxml.html
import os.path
import zipfile
import glob
from collections import OrderedDict
import json

In [123]:
base = 'http://www.psp.cz/eknih/2013ps/stenprot/zip/index.htm'
zp = './zip/'
st = './html/'

Stenoprotokoly jsou zazipované, ale jsou stále ve spousta balících. Stáhneme tedy stránku, na které jsou, projdeme odkazy na zipy a stáhneme ty.

In [124]:
ht = lxml.html.parse(base).getroot()

In [125]:
if not os.path.exists('zip'): os.mkdir('zip')
for ln in ht.cssselect('div#main-content a'):
    tfn = os.path.join(zp, ln.attrib['href'])
    if os.path.isfile(tfn): continue
    print(ln.attrib['href'])
    ur.urlretrieve(up.urljoin(base, ln.attrib['href']), tfn)

059schuz.zip


HTTPError: HTTP Error 404: Not Found

Rozbalíme (šlo by číst přímo z archivů, ale proč si to komplikovat, když to místa zabírá málo)

In [126]:
if not os.path.exists('html'): os.mkdir('html')

fns = glob.glob(os.path.join(zp, '*.zip'))

for fn in fns:
    zfn = os.path.split(fn)
    zfn = zfn[-1].replace('.zip', '')
    pth = os.path.join(st, zfn)
    if not os.path.exists(pth): os.mkdir(pth)

    with zipfile.ZipFile(fn) as zf:
        zf.extractall(pth)

Převedeme do textu. Není tu moc rocket science, vesměs jde o:

- Každý odstavec je buďto pokračování projevu, nebo začátek nového.
- Začátek projevu poznáme tak, že začíná odkazem na profil řečníka. Hledat odkaz v textu je ale celkem riskantní, protože některé projevy (zvlášť v minulé Sněmovně) obsahovaly odkazy na hlasování. Tohle bude třeba ještě nějak ošetřit.
- Proslovy mohou přetékat mezi soubory. Na začátku souboru je tak třeba zjistit, zda jde o pokračování, nebo ne. Já to řeším tak, že konce souborů neřeším, celá jedna schůze je pro mě tok textu, bez ohledu na rozdělení mezi soubory.
- Jména řečníků jsou vč. jejich funkcí. To je nepříjemné ze dvou důvodů: jméno je moc dlouhé, zvlášť u facetování (filtrace v hledání), ale hlavně protože jméno pak není unikátní, protože funkce se mění, zvlášť mezi sněmovnami. Když pak budeme chtít seskupit spoustu let dohromady, budeme mít např. Bohuslava Sobotku jako ministra financí, poslance a premiéra. Bohužel rozdělení jména na funkce a vlastní jméno není úplně jednoznačné, bude to chtít vypsat počet variací a ošetřit to. Časem.

In [127]:
jsp = 'json/'
if not os.path.exists(jsp): os.mkdir(jsp)
schuze = glob.glob('./html/*')

In [154]:
poz = 'Poslanec; Místopředsedkyně PSP; Místopředseda PSP; Předseda PSP; Paní; Senátorka; Senátor; \
Poslankyně; Evropská komisařka pro spravedlnost, spotřebitele a rovnost žen a mužů; \
Předsedající; Veřejná ochránkyně práv; Prezident České republiky; \
Místopředseda vlády ČR a ministr financí; Místopředseda vlády ČR a ministr životního prostředí'.split('; ')

def depozicuj(jmeno):
    for p in poz:
        if jmeno.startswith(p):
            return jmeno[len(p)+1:]
            
    if 'ČR' in jmeno:
        return jmeno[jmeno.index('ČR') + 3:]
    
    return jmeno

In [157]:
def zapisuj(nm, dt):
    fn = os.path.join(jsp, '%s.json' % nm)
    
    with open(fn, 'w') as f:
        t = json.dump(dt, f, ensure_ascii=False, indent=2)

res = []
pid = 0
aut = None
tema = None
buf = []
for sch in schuze:
    schd = []
    fns = glob.glob(os.path.join(sch, 's*.htm'))
    for fn in fns:
        h = lxml.html.parse(fn).getroot()
        for p in h.cssselect('p'):
            pt = p.text_content().strip()
            if len(pt) == 0: continue
            pt = pt.replace('\xa0', ' ')
            
            od = p.find('a') # v textu je odkaz
            if od is None:
                buf += [pt]
                continue

            if len(buf) > 0:
                schd.extend([OrderedDict(id=pid, autor=aut, schuze=int(sch.split('/')[-1][:3]),\
                                         fn=fn, tema=tema, text='\n'.join(buf))])

            aut = depozicuj(od.text.strip())
            buf = [pt[len(od.text)+1:].strip()] # pridame soucasny text (ale odseknem autora)
            pid += 1
    zapisuj(sch[-sch[::-1].find('/'):], schd)

print(sch)

./html/058schuz


## depozicování kontrola

In [158]:
from collections import Counter

In [159]:
fns = glob.glob('json/*.json')

In [160]:
auts = []
for fn in fns:
    with open(fn) as f:
        dt = json.load(f)
    
    for el in dt:
        if el['autor'] is not None:
            auts.append(el['autor'])

In [161]:
Counter(auts).most_common()[:3]

[('Petr Gazdík', 8814), ('Vojtěch Filip', 8547), ('Jan Bartošek', 7539)]

In [162]:
[j for j in set(auts) if len(j.split(' ')) > 2]

['Zuzka Bebarová Rujbrová',
 'Augustin Karel Andrle Sylor',
 'Jaroslava Pokorná Jermanová',
 'Markéta Pekarová Adamová',
 'Hana Aulická Jírovcová',
 'Tomáš Jan Podivínský']

Tak, hotovo. Můžeme sypat do hledače.